In [1]:
pwd

'C:\\Users\\Sneha\\Desktop\\bse_scraper'

---------------------------------------------------------------------------------------------------------------------------

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [3]:
import time
import string

In [4]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [12]:

def scrape_one_stock(stock_name, op , url):
    driver = webdriver.Chrome(executable_path='D:\chromedriver_win32\chromedriver.exe', chrome_options=op )

    driver.get(url)

    # Put values for "Security Name"

    e1 = driver.find_element_by_xpath('//*[@id="scripsearchtxtbx"]').click()
    action = ActionChains(driver)
    action.send_keys(stock_name)

    driver.implicitly_wait(30)

    action.send_keys(Keys.ENTER)
    driver.implicitly_wait(10)
    action.send_keys(Keys.RETURN)
    action.perform()


    # Select "Daily" Radio Button

    e2 = driver.find_element_by_xpath('//*[@id="rdbDaily"]').click()


    # FROM 

    action2 = ActionChains(driver)

    fromfield = driver.find_element_by_xpath('//*[@id="txtFromDate"]')

    driver.execute_script("arguments[0].removeAttribute('onkeypress')",fromfield )
    driver.execute_script("arguments[0].setAttribute('value','01/01/2019')", fromfield)
    action2.send_keys(Keys.RETURN).perform()

    driver.implicitly_wait(10)

    # TO
    action3 = ActionChains(driver)

    tofield = driver.find_element_by_xpath('//*[@id="txtToDate"]')

    driver.execute_script("arguments[0].removeAttribute('onkeypress')", tofield)
    driver.execute_script("arguments[0].removeAttribute('disabled')", tofield)
    driver.execute_script("arguments[0].setAttribute('value','30/01/2022')", tofield)
    action3.send_keys(Keys.RETURN).perform()

    # SUBMIT

    action4 = ActionChains(driver)
    sub_btn = driver.find_element_by_xpath('//*[@id="btnSubmit"]')
    action4.move_to_element(sub_btn).click().click().perform()


    driver.implicitly_wait(10)


    ##################

    down = driver.find_element_by_xpath('//*[@id="lnkDownload"]')

    print(down.is_displayed())

    if(down.is_displayed()):
        down.click()
        
    driver.close()

In [11]:
# options to download in a particular file, not in Downloads folder
op = webdriver.ChromeOptions()
p = {"download.default_directory": r"C:\Users\Sneha\Desktop\bse_scraper\scraped_csv_files" , "safebrowsing.enabled":"false"}
op.add_experimental_option("prefs", p)

# main url 
url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.html?flag=0"

# list of names of stocks to be scraped
l = ['TATA MOTORS LTD', 'ICICI BANK LTD', 'PVR LTD', 'DRREDDYS LABORATORIES LTD', 'EXIDE INDUSTRIES LTD']

for i in l:
    scrape_one_stock(i, op, url)

C:\Users\Sneha\anaconda3\envs\scraper\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


True
True
True
True
True



### CSV files successfully extracted.

In [13]:
import pandas as pd

In [16]:
# making a list of names of all the scraped csv files

from os import listdir

def find_csv_files( path, suffix=".csv" ):
    all_files = listdir(path)
    return [ csvfile for csvfile in all_files if csvfile.endswith( suffix ) ]

f = find_csv_files( r"C:\Users\Sneha\Desktop\bse_scraper\scraped_csv_files")

print(f)

['500086.csv', '500124.csv', '500570.csv', '532174.csv', '532689.csv']


In [ ]:
def append_csv( final, single):
    """ to stack dataframes vertically over one another"""
    

In [52]:
base_path = "C:\\Users\\Sneha\\Desktop\\bse_scraper\\scraped_csv_files\\" 

# reading first csv

final_df = pd.read_csv(base_path + f[0])

print("before : " + str(len(final_df)))


# appending all other csv files UNDER it

for i in range(1,len(f)) :
    fname = base_path + f[i]
    
    temp_df = pd.read_csv(fname)
    
    final_df = pd.concat( [final_df, temp_df], axis=0)
    
print("after : " + str(len(final_df)))
    

before : 764
after : 3820


In [54]:
final_df.tail(6)

,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
758,8-January-2019,1576.40,1610.35,1576.40,1604.70,1600.917989,10023,478,16046001.0,1817,18.13,33.95,28.30
759,7-January-2019,1575.55,1596.35,1575.55,1580.90,1585.494114,11552,607,18315628.0,1243,10.76,20.80,5.35
760,4-January-2019,1540.00,1577.80,1502.00,1573.50,1543.545928,29960,1398,46244636.0,2561,8.55,75.80,33.50
761,3-January-2019,1587.00,1611.00,1518.50,1532.35,1552.504154,36352,1884,56436631.0,2205,6.07,92.50,-54.65
762,2-January-2019,1582.80,1602.40,1578.10,1588.90,1590.930576,11509,566,18310020.0,693,6.02,24.30,6.10
763,1-January-2019,1598.35,1601.30,1580.80,1593.90,1590.907211,5367,331,8538399.0,429,7.99,20.50,-4.45


The index column here is repeated for each dataset portion (0-763), so we need to change the index so that it corresponds to row number in the final dataframe, that is, (0-3819)

In [56]:
final_df.reset_index(inplace=True)

In [57]:
final_df.tail(4)

,index,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
3816,760,4-January-2019,1540.00,1577.8,1502.0,1573.50,1543.545928,29960,1398,46244636.0,2561,8.55,75.8,33.50
3817,761,3-January-2019,1587.00,1611.0,1518.5,1532.35,1552.504154,36352,1884,56436631.0,2205,6.07,92.5,-54.65
3818,762,2-January-2019,1582.80,1602.4,1578.1,1588.90,1590.930576,11509,566,18310020.0,693,6.02,24.3,6.10
3819,763,1-January-2019,1598.35,1601.3,1580.8,1593.90,1590.907211,5367,331,8538399.0,429,7.99,20.5,-4.45


Next, dropping the old index column, as it has served its purpose.

In [61]:
final_df.drop(columns='index', axis=0, inplace=True)

In [62]:
# final dataframe 

final_df.tail(6)

,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
3814,8-January-2019,1576.40,1610.35,1576.40,1604.70,1600.917989,10023,478,16046001.0,1817,18.13,33.95,28.30
3815,7-January-2019,1575.55,1596.35,1575.55,1580.90,1585.494114,11552,607,18315628.0,1243,10.76,20.80,5.35
3816,4-January-2019,1540.00,1577.80,1502.00,1573.50,1543.545928,29960,1398,46244636.0,2561,8.55,75.80,33.50
3817,3-January-2019,1587.00,1611.00,1518.50,1532.35,1552.504154,36352,1884,56436631.0,2205,6.07,92.50,-54.65
3818,2-January-2019,1582.80,1602.40,1578.10,1588.90,1590.930576,11509,566,18310020.0,693,6.02,24.30,6.10
3819,1-January-2019,1598.35,1601.30,1580.80,1593.90,1590.907211,5367,331,8538399.0,429,7.99,20.50,-4.45


### Pickling the final dataframe

In [63]:
import pickle

In [67]:
# new pickle file
picklefile = open('final_pickle', 'wb')

In [68]:
# picking the final df
pickle.dump(final_df, picklefile)

In [69]:
picklefile.close()